In [20]:
import table
import table.IO as tio
import torch

import pandas as pd

ImportError: No module named 'pandas'

In [19]:
base_dir = '../data_model/conala/'
conala_raw = '../../code-gen.git/raw-datasets/conala-corpus/'



{'eos_token': None, 'sequential': True, 'init_token': None, 'lower': False, 'tensor_type': <class 'torch.LongTensor'>, 'fix_length': None, 'include_lengths': True, 'pad_token': '<blank>', 'batch_first': False, 'postprocessing': None, 'use_vocab': True, 'unk_token': '<unk>', 'tokenize': <function Field.<lambda> at 0x7ff000b4ac80>, 'preprocessing': None}
